In [1]:
import pandas as pd
import gzip
import pickle
import matplotlib.pyplot as plt
import cx_Oracle 
import numpy as np

In [3]:
conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
cur = conn.cursor()

In [ ]:
ls = '2014-12-01'

In [6]:
query = "SELECT store_name FROM STORE_V WHERE MODIFIED_DATE BETWEEN to_date('12/feb/2014','DD/MM/YYYY') AND to_date('05/May/2016','DD/MM/YYYY')"

In [6]:
query = 'select v.invc_sid, v.sbs_no, v.store_no, v.invc_type, v.disc_amt, v.created_date, v.trans_disc_amt, l.redeemed_amt, l.total_based_disc, perc_reward_disc_amt, (l.used_points_positive+l.used_points_negative) as Loyalty_used, s.store_name, i.qty, (i.qty*price) as ExtPrice, i.price, i.cost, (i.qty*i.cost) as ExtCost, i.tax_amt, i.tax_amt2, i.usr_disc_perc, t.amt, t.tender_type from invc_item_v  i left join Invoice_v v on i.invc_sid= v.invc_sid left join invc_lty_v l on v.invc_sid = l.invc_sid left join  invc_tender_v t on v.invc_sid = t.invc_sid left join store_v s on (v.sbs_no+v.store_no)  = (s.sbs_no+ s.store_no)'

In [7]:
cur.execute(query)

<cx_Oracle.Cursor on <cx_Oracle.Connection to ReportUser@10.10.1.7:1521/rproods>>

In [42]:
def getRecords(date1, date2):
    query = "select v.invc_sid, v.sbs_no, v.store_no, v.invc_type, v.disc_amt, v.created_date, v.trans_disc_amt, l.redeemed_amt, l.total_based_disc, perc_reward_disc_amt, (l.used_points_positive+l.used_points_negative) as Loyalty_used, s.store_name, i.qty, (i.qty*price) as ExtPrice, i.price, i.cost, (i.qty*i.cost) as ExtCost, i.tax_amt, i.tax_amt2, i.usr_disc_perc, t.amt, t.tender_type from invc_item_v  i left join Invoice_v v on i.invc_sid= v.invc_sid left join invc_lty_v l on v.invc_sid = l.invc_sid left join  invc_tender_v t on v.invc_sid = t.invc_sid left join store_v s on (v.sbs_no+v.store_no)  = (s.sbs_no+ s.store_no) where v.created_date BETWEEN to_date('{}','DD/MM/YYYY') AND to_date('{}','DD/MM/YYYY') ".format(date1,date2)
    cur.execute(query)
    rec = cur.fetchall()
    columns = ['invc_sid', 'subsidiary', 'store_no', 'invc_type', 'disc_amt', 'created_date', 'trans_disc_amt', 'lyt_redeemed_amt', 'lyt_total_based_disc', 'lyt_perc_reward_disc_amt', 'Loyalty_used', 'store_name', 'item_qty', 'ExtPrice', 'unit_price', 'unit_cost', 'ExtCost', 'tax_amt', 'tax_amt2', 'usr_disc_perc', 'paid_amt', 'tender_type']
    data = pd.DataFrame(rec, columns=columns)
    return data
    
    

In [43]:
k = getRecords('05/May/2016', '31/May/2016')

In [44]:
k.drop('tax_amt2',axis =1,  inplace= True)

In [51]:
k.fillna(0, inplace=True)

In [78]:
k.head()

,invc_sid,subsidiary,store_no,invc_type,disc_amt,created_date,trans_disc_amt,lyt_redeemed_amt,lyt_total_based_disc,lyt_perc_reward_disc_amt,...,store_name,item_qty,ExtPrice,unit_price,unit_cost,ExtCost,tax_amt,usr_disc_perc,paid_amt,tender_type
0,-401991826868338024,100,36,0,0.0,2016-05-06 17:36:19,520.0,0.0,520.0,0.0,...,HealthPlus Town Planning,1.0,1020.0,1020.0,722.2900,722.2900,0.0,0.0,500.0,0
1,-401976460684558994,100,13,0,0.0,2016-05-06 17:36:30,0.0,0.0,0.0,0.0,...,HealthPlus Ikeja City Mall,1.0,370.0,370.0,275.1454,275.1454,0.0,0.0,370.0,0
2,-401964513581795150,100,20,0,0.0,2016-05-06 17:36:39,0.0,0.0,0.0,0.0,...,HealthPlus Ceddi Plaza,1.0,190.0,190.0,152.4800,152.4800,0.0,0.0,200.0,0
3,-401964513581795150,100,20,0,0.0,2016-05-06 17:36:39,0.0,0.0,0.0,0.0,...,HealthPlus Ceddi Plaza,1.0,190.0,190.0,152.4800,152.4800,0.0,0.0,-10.0,5
4,-401977093258578027,100,41,0,0.0,2016-05-06 17:36:52,0.0,0.0,0.0,0.0,...,HealthPlus Akure,1.0,770.0,770.0,504.4121,504.4121,0.0,0.0,770.0,0


In [65]:
sales = k.groupby(['invc_sid'])['ExtPrice','ExtCost', 'disc_amt','trans_disc_amt','lyt_redeemed_amt','lyt_total_based_disc', 'lyt_perc_reward_disc_amt', 'Loyalty_used', 'tax_amt','usr_disc_perc'].sum()

In [69]:
sales.head(50)

,ExtPrice,ExtCost,disc_amt,trans_disc_amt,lyt_redeemed_amt,lyt_total_based_disc,lyt_perc_reward_disc_amt,Loyalty_used,tax_amt,usr_disc_perc
invc_sid,,,,,,,,,,
-464990732433242710,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
-407309465213890499,3660.0,2213.7500,0.0,0.0,0.0,0.0,0.0,0.0,193.00200,0.0
-407297245628500289,1920.0,1368.8240,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
-407294796539294370,2650.0,1318.5119,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
-407294666622931486,1550.0,933.8000,0.0,50.0,0.0,50.0,0.0,50.0,0.00000,0.0
-407288805855440868,480.0,301.3830,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
-407274570940301114,3360.0,2457.4880,0.0,600.0,0.0,600.0,0.0,600.0,0.00000,0.0
-407262985698123316,2400.0,1494.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
-407257886037346747,280.0,171.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [70]:
no_invc_dup = k.drop_duplicates('invc_sid')

In [82]:
sales = sales.merge(no_invc_dup[['invc_sid','subsidiary', 'store_no', 'tender_type', 'store_name', 'paid_amt']], on='invc_sid')

In [83]:
sales.head()

,invc_sid,ExtPrice,ExtCost,disc_amt,trans_disc_amt,lyt_redeemed_amt,lyt_total_based_disc,lyt_perc_reward_disc_amt,Loyalty_used,tax_amt,usr_disc_perc,tender_type_x,store_name_x,paid_amt_x,subsidiary,store_no,tender_type_y,store_name_y,paid_amt_y
0,-464990732433242710,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,HealthPlus Domino,0.0,100,7,0,HealthPlus Domino,0.0
1,-407309465213890499,3660.0,2213.7500,0.0,0.0,0.0,0.0,0.0,0.0,193.002,0.0,0,HealthPlus MMA1,3855.0,100,5,0,HealthPlus MMA1,3855.0
2,-407297245628500289,1920.0,1368.8240,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,HealthPlus Lekki 2,1920.0,100,40,0,HealthPlus Lekki 2,1920.0
3,-407294796539294370,2650.0,1318.5119,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0,HealthPlus Allen,2650.0,100,11,0,HealthPlus Allen,2650.0
4,-407294666622931486,1550.0,933.8000,0.0,50.0,0.0,50.0,0.0,50.0,0.000,0.0,0,HealthPlus Lekki,1500.0,100,15,0,HealthPlus Lekki,1500.0


In [47]:
p = k.groupby(['subsidiary','store_no']).sum()

In [46]:
p

invc_sid  invc_type  disc_amt  trans_disc_amt  \
subsidiary store_no                                                      
100        0         1.154340e+21      370.0   1993.00          949.00   
           2        -1.751498e+21        0.0      0.00       352880.50   
           3        -2.071012e+21        0.0      0.00       359827.10   
           4        -1.191565e+21        0.0      0.00       291813.00   
           5        -1.478906e+21        0.0     12.22       224233.27   
           6        -1.145968e+21        0.0      0.00       120976.00   
           7        -9.252333e+20        0.0      0.00       112192.00   
           8        -1.553394e+21        0.0      0.00       208380.00   
           9        -1.635994e+21        0.0      0.00       271194.00   
           10       -1.600336e+21        0.0      0.00       308729.00   
           11       -1.901107e+21        0.0      0.00       300584.00   
           12       -1.146942e+21        0.0      0.00       123014.00   
           13       -2.572237e+21        0.0      0.00       489919.00   
           14       -8.474626e+20        0.0      0.00       149774.00   
           15       -2.545443e+21        0.0      0.00       520712.60   
           16       -1.528344e+21        0.0      0.00       213457.50   
           17       -8.631346e+20        0.0      0.00        53633.50   
           18       -8.317825e+20        0.0      0.00        84642.00   
           19       -1.127345e+21        0.0      0.00       143162.00   
           20       -7.234108e+20        0.0      0.00        99999.00   
           21       -1.199086e+21        0.0      0.00       149108.00   
           22       -6.030259e+20        0.0      0.00        62301.00   
           23       -8.380020e+20        0.0      0.00        69815.00   
           24       -6.760598e+20        0.0      0.00        63528.00   
           25       -1.262364e+21        0.0      0.00        70258.00   
           26       -8.747711e+20        0.0      0.00        80601.00   
           27       -6.759530e+20        0.0      0.00        59783.40   
           28       -5.297147e+20        0.0      0.00        84402.00   
           29       -5.500244e+20        0.0      0.00        40466.00   
           30       -7.271999e+20        0.0      0.00        38508.00   
...                           ...        ...       ...             ...   
           41       -6.455463e+20        0.0      0.00        21543.00   
           43       -8.067133e+20        0.0      0.00       115876.00   
           44       -9.012572e+20        0.0      0.00        53527.00   
           46       -1.587186e+21        0.0      0.00       156273.50   
           47       -5.919967e+20        0.0      0.00        63106.00   
200        0         3.785644e+19       12.0      0.00            0.00   
           2        -1.340250e+21        0.0      0.00       324224.00   
           3        -5.624156e+20        0.0      0.00       125970.00   
           4        -1.097946e+21        0.0      0.00       176152.00   
           5        -6.161958e+20        0.0      0.00       113846.00   
           6        -8.275065e+20        0.0      0.00       230949.40   
           7        -5.504847e+20        0.0      0.00       353585.80   
           8        -7.392989e+20        0.0      0.00       209264.80   
           9        -4.430727e+20        0.0      0.00       161953.00   
           10       -3.945658e+20        0.0   2645.00       117541.00   
           11       -4.265965e+20        0.0      0.00        42516.00   
           12       -7.992487e+20        0.0      0.00       122038.00   
           13       -2.913688e+20        0.0      0.00        33749.00   
           14       -3.438158e+20        0.0      0.00        73775.50   
           15       -4.492629e+20        0.0      0.00        72459.70   
           16       -4.417883e+20        0.0      0.00        50348.00   
           17       -7.547792

In [ ]:
with gzip.open('cus_records.gzip', 'wb') as f:
    data = pickle.dump(d,f)
    

In [ ]:
data.to_csv('cust_sales_records.csv')

In [ ]:
pd.read_csv?

In [ ]:
!ls

In [ ]:
d =  pd.read_csv('cust_sales_records.csv')

In [ ]:
d.sort_values('CREATED_DATE.1').hail(5)

In [ ]:
d.drop(columns = 'INVC_SID.1', inplace=True)

In [ ]:
d.sort_values?


In [ ]:
last_date = d.sort_values('CREATED_DATE.1', ascending=True).dropna(subset=['CREATED_DATE.1']).tail(1)

In [ ]:
last_date.to_datetime(['CREATED_DATE'])

In [ ]:
last = pd.to_datetime(last_date['CREATED_DATE'])

In [ ]:
last

In [ ]:
q = d


In [ ]:
q.dropna(subset=['CREATED_DATE.1'])